In [24]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv("reviews3.csv")
df.head()

,"good rates,quick @easy,only took minutes","Date of experience: March 27, 2025"
0,"good rates,quick @easy,only took minutes","Date of experience: March 27, 2025"
1,Fast easy service....good prices.,"Date of experience: March 26, 2025"
2,Great service,"Date of experience: March 26, 2025"
3,No waiting time to reach a representative,No waiting time to reach a representative.\nAn...
4,Superb service and rates better than most places,We have dealt with snowbird advisor insurance ...


In [3]:
df = df.rename(columns={'good rates,quick @easy,only took minutes': 'title', 'Date of experience: March 27, 2025': 'review'})

In [4]:
df.head()

,title,review
0,"good rates,quick @easy,only took minutes","Date of experience: March 27, 2025"
1,Fast easy service....good prices.,"Date of experience: March 26, 2025"
2,Great service,"Date of experience: March 26, 2025"
3,No waiting time to reach a representative,No waiting time to reach a representative.\nAn...
4,Superb service and rates better than most places,We have dealt with snowbird advisor insurance ...


In [5]:
count = df['review'].str.startswith("Date of experience").sum()
print(count)

243


In [6]:
condition = df['review'].str.startswith("Date of experience")
df.loc[condition, 'review'] = df.loc[condition, 'title']

In [7]:
df.head()

,title,review
0,"good rates,quick @easy,only took minutes","good rates,quick @easy,only took minutes"
1,Fast easy service....good prices.,Fast easy service....good prices.
2,Great service,Great service
3,No waiting time to reach a representative,No waiting time to reach a representative.\nAn...
4,Superb service and rates better than most places,We have dealt with snowbird advisor insurance ...


In [8]:
df.drop(columns=['title'], inplace=True)

In [9]:
df.head()

,review
0,"good rates,quick @easy,only took minutes"
1,Fast easy service....good prices.
2,Great service
3,No waiting time to reach a representative.\nAn...
4,We have dealt with snowbird advisor insurance ...


In [10]:
column_with_reviews = 'review'
rows_with_newlines = df[df[column_with_reviews].str.contains('\n') | df[column_with_reviews].str.contains('\r\n')]
print("Rows with newline characters:")
print(rows_with_newlines)

Rows with newline characters:
                                                 review
3     No waiting time to reach a representative.\nAn...
4     We have dealt with snowbird advisor insurance ...
16    We have trusted Snowbird out of country health...
19    People on the phone very helpful and knowledge...
30                       Great customer service\nPrices
...                                                 ...
1930  Great price, but not very clear on which one t...
1940  Website is easy to understand and follow.\nPre...
1974  Knee replacement resulted in extra insurance c...
1995  They were very helpful and curtious,also expla...
1996  Very unclear about "existing conditions"?\nDo ...

[202 rows x 1 columns]


In [11]:
df[column_with_reviews] = df[column_with_reviews].str.replace('\n', ' ', regex=True)
df[column_with_reviews] = df[column_with_reviews].str.replace('\r\n', ' ', regex=True)

In [12]:
def remove_non_letters(text):
    return re.sub(r'[^a-zA-Z]', ' ', text)

df['review'] = df['review'].apply(remove_non_letters)

In [13]:
column_with_reviews = 'review'
df[column_with_reviews] = df[column_with_reviews].str.replace('“', '"', regex=False)  # Replace opening curly quote
df[column_with_reviews] = df[column_with_reviews].str.replace('”', '"', regex=False)  # Replace closing curly quote
df[column_with_reviews] = df[column_with_reviews].str.replace('\'', '', regex=True)   
df[column_with_reviews] = df[column_with_reviews].str.replace('\"', '', regex=True)
df[column_with_reviews] = df[column_with_reviews].str.replace('\(', '', regex=True)  
df[column_with_reviews] = df[column_with_reviews].str.replace('\)', '', regex=True)  

<>:6: SyntaxWarning: invalid escape sequence '\('
<>:7: SyntaxWarning: invalid escape sequence '\)'
<>:6: SyntaxWarning: invalid escape sequence '\('
<>:7: SyntaxWarning: invalid escape sequence '\)'
C:\Users\HP\AppData\Local\Temp\ipykernel_21380\303443749.py:6: SyntaxWarning: invalid escape sequence '\('
  df[column_with_reviews] = df[column_with_reviews].str.replace('\(', '', regex=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_21380\303443749.py:7: SyntaxWarning: invalid escape sequence '\)'
  df[column_with_reviews] = df[column_with_reviews].str.replace('\)', '', regex=True)


In [14]:
df.to_csv("review_preprocessed.csv", index=False)

In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [16]:
obj = SentimentIntensityAnalyzer()

In [17]:
def get_sentiment_score(text):
    # Get the sentiment scores for the text
    sentiment_scores = obj.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    
    # Map the compound score to discrete values
    if compound_score > 0.05:
        return 1  # Positive
    elif compound_score < -0.05:
        return -1  # Negative
    else:
        return 0  # Neutral

In [18]:
df['score'] = df['review'].apply(get_sentiment_score)

In [19]:
df.head()

,review,score
0,good rates quick easy only took minutes,1
1,Fast easy service good prices,1
2,Great service,1
3,No waiting time to reach a representative And...,1
4,We have dealt with snowbird advisor insurance ...,1


In [20]:
columns = df.columns.tolist()

# Swap the first and second columns
columns[0], columns[1] = columns[1], columns[0]

# Reorder the DataFrame
df = df[columns]

In [21]:
df.to_csv("df_with_score.csv", index=False)

In [32]:
df_new = pd.read_csv("df_with_score_visualization.csv")

In [33]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [34]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

df_new['processed'] = df_new['review'].apply(preprocess_text)

print(df_new[['review', 'processed']].head())

                                              review  \
0           good rates quick  easy only took minutes   
1                  Fast easy service    good prices    
2                                      Great service   
3  No waiting time to reach a representative  And...   
4  We have dealt with snowbird advisor insurance ...   

                                           processed  
0            [good, rate, quick, easy, took, minute]  
1                 [fast, easy, service, good, price]  
2                                   [great, service]  
3  [waiting, time, reach, representative, represe...  
4  [dealt, snowbird, advisor, insurance, several,...  


In [38]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.matutils import corpus2dense

In [ ]:
dictionary = Dictionary(df_new['processed'])

dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in df_new['processed']]

In [ ]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=10)

topics = lda_model.print_topics(num_words=10)
for idx, topic in enumerate(topics):
    print(f"Topic {idx + 1}: {topic}")

Topic 1: (0, '0.027*"insurance" + 0.025*"helpful" + 0.025*"knowledgeable" + 0.024*"representative" + 0.024*"professional" + 0.018*"good" + 0.017*"question" + 0.017*"friendly" + 0.015*"easy" + 0.015*"well"')
Topic 2: (1, '0.072*"service" + 0.061*"great" + 0.040*"easy" + 0.034*"price" + 0.031*"customer" + 0.027*"friendly" + 0.026*"good" + 0.022*"fast" + 0.022*"efficient" + 0.021*"excellent"')
Topic 3: (2, '0.117*"easy" + 0.041*"insurance" + 0.031*"quick" + 0.028*"use" + 0.028*"price" + 0.025*"online" + 0.022*"travel" + 0.019*"quote" + 0.019*"process" + 0.019*"snowbird"')
Topic 4: (3, '0.054*"helpful" + 0.050*"agent" + 0.040*"question" + 0.029*"phone" + 0.022*"knowledgeable" + 0.021*"answered" + 0.020*"person" + 0.019*"pleasant" + 0.018*"patient" + 0.016*"friendly"')
Topic 5: (4, '0.049*"insurance" + 0.026*"travel" + 0.018*"time" + 0.016*"helpful" + 0.016*"company" + 0.015*"question" + 0.014*"health" + 0.014*"policy" + 0.014*"u" + 0.014*"knowledgeable"')


In [42]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Prepare the visualization
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization
pyLDAvis.display(vis_data)

c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\jobli